# Experiments
Having implemented the different networks and models we can now run them and analyse the results of different parameterisations. This is meant as an exploratory analysis of the effects of different parameters. For the web app we will simualate the models with parameters on a finer grid.

In [1]:
# allow relative imports
import os
import sys

module = os.path.abspath(os.path.join('..'))
if module not in sys.path:
    sys.path.append(module)
    

# utils
from functools import partial
    
# reproducibility
SEED = 4401

The following Human Contact Networks will be compared:
- Mobility network
- Power law with cutoff
- Physical distancing network

For each, we will evaluate the following epidemic models:
- SEIR
- SEIVR
- SEIR with quarantine
- SEVIR with quarantine

Each model will require slightly different parameters.

## Networks

For each of the three types of networks we will create two instances: a baseline network representing nodes living under no restrictions/distancing etc. (we will call this __pre__) and a network assuming some form of physical distancing put in place (called __post__). We can then plug in each of these networks into the different models.

The mobility networks have `10000` nodes and average node degrees of `4` and `3` for __pre__ and __post__ respectively. Where appropriate, we will mimick these characteristics in the other networks for comparability reasons.

### Mobility network

The mobility networks are based on mobility data from SafeGraph for New Haven County, CT, USA. The __pre__ network is adapted from data from February 2020, before stay-at-home orders were issued, while the __post__ network is based on data from April 2020, after such orders were put in place in March.

The graphs for the two networks have already been producen (see *network_creation.ipynb*), so we can use the `MNGeneratorFromFile`. For now, it suffices defining the parameters of the generator, which we can then pass to the experiment later on.

In [2]:
from data_processing.data_paths import GRAPHS
from model.network.mobility_network import MNGeneratorFromFile as MNGF

In [3]:
params_mobility_pre = dict()
params_mobility_pre[MNGF.PATH] = os.path.join(GRAPHS, 'pre_N_10000_k_3.graphml')

params_mobility_post = dict()
params_mobility_post[MNGF.PATH] = os.path.join(GRAPHS, 'post_N_10000_k_3.graphml')

### Power law with cutoff

The power law with cutoff (PLC) creates a network with nodes with degrees that follow a power law distribution but have a cutoff point at the upper end to prevent implausibly high degrees. `epydemic` provides a NetworkGenerator for this type of network so we can create it from there. The cutoff parameter is set to `10`. Again, we only define the parameters now and pass them to the experiment with the generator later.

In [4]:
from epydemic import PLCNetwork

In [5]:
params_plc_pre = dict()
params_plc_pre[PLCNetwork.N] = 10000
params_plc_pre[PLCNetwork.EXPONENT] = 4
params_plc_pre[PLCNetwork.CUTOFF] = 10

params_plc_post = dict()
params_plc_post[PLCNetwork.N] = 10000
params_plc_post[PLCNetwork.EXPONENT] = 3
params_plc_post[PLCNetwork.CUTOFF] = 10

### Physical distancing

In [6]:
from model.network.distanced_network import DNGenerator as DNG
from model.distributions import household_size_dist, intra_household_contacts, node_degree_dist

In [7]:
average_family = partial(household_size_dist, mu=4.5, std=2)
intra_family_contacts = partial(intra_household_contacts, std=1)
inter_family_contacts = node_degree_dist

params_distanced_pre = dict()
params_distanced_pre[DNG.N] = 10000
params_distanced_pre[DNG.CLUSTER_SIZE_DIST] = average_family
params_distanced_pre[DNG.CONTACT_DIST] = intra_family_contacts
params_distanced_pre[DNG.CLUSTER_CONTACT_DIST] = partial(inter_family_contacts, exponent=4)
params_distanced_pre[DNG.SEED] = SEED

params_distanced_post = dict()
params_distanced_post[DNG.N] = 10000
params_distanced_post[DNG.CLUSTER_SIZE_DIST] = average_family
params_distanced_post[DNG.CONTACT_DIST] = intra_family_contacts
params_distanced_post[DNG.CLUSTER_CONTACT_DIST] = partial(inter_family_contacts, exponent=3)
params_distanced_post[DNG.SEED] = SEED

## Epidemic model